<a href="https://colab.research.google.com/github/AnnSenina/Python_CL_2023/blob/main/notebooks/Python_12_pymorphy%2C_mystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лемматизация, морф.анализ

In [ ]:
#!pip install nltk
#!pip install pymorphy2
#!pip install pymystem3

In [3]:
# все импорты...

import nltk
from nltk.tokenize import word_tokenize
from nltk import download
download('punkt')
download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

from collections import Counter

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

from pymystem3 import Mystem
mystem = Mystem()

Для русского основых лемматизатора два: Pymorphy и Mystem.

##PyMorphy2

Очень популярный и простой инструмент для лемматизации русского языка

In [ ]:
!pip install pymorphy2

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

### основная функция - pymorphy.parse
     
Она похожа на analyze в mystem, возвращает список объектов Parse

Первый в списке - самый вероятный разбор (у каждого есть score)

Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать

In [ ]:
morph.parse('человек')

Документация [здесь](https://pymorphy2.readthedocs.io/en/stable/user/guide.html#id3), обозначения для граммем [здесь](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs)

In [ ]:
# сделаем красиво
print('Cлово - ', morph.parse('человеком')[0].word)
print('Лемма слова - ', morph.parse('человеком')[0].normal_form)
print('Грамматическая информация слова - ', morph.parse('человеком')[0].tag)
print('Часть речи слова - ', morph.parse('человеком')[0].tag.POS)
print('Род слова - ', morph.parse('человеком')[0].tag.gender)
print('Число слова - ', morph.parse('человеком')[0].tag.number)
print('Падеж слова - ', morph.parse('человеком')[0].tag.case)

In [ ]:
# для лемматизации нужна именно эта команда
morph.parse('человеком')[0].normal_form

Давайте возьмем Сэлинджера...

Увы! pymorphy не умеет токенизировать, поэтому...

In [ ]:
# препроцессинг в 1 функцию (старая функция)
def clean_text(text_str):
  text_str = text_str.lower() # нижний регистр
  text_list_nltk = word_tokenize(text_str) # токенизация
  stop_words = stopwords.words('russian')
  text_clean = [word for word in text_list_nltk if word not in stop_words and word[0].isalpha()] # чистим от стоп-слов и пунктуации
  return text_clean

with open('bananafish.txt', 'r', encoding='utf-8') as file:
  bananafish = file.read()
bananafish_tokens = clean_text(bananafish)
bananafish_tokens
# print(bananafish_tokens)

In [ ]:
words_lemmatized = []
for word in bananafish_tokens:
    result = morph.parse(word)
    most_probable_result = result[0] ## почему мы берем первый разбор? см.в этом месте: https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct
    normal_form = most_probable_result.normal_form
    words_lemmatized.append(normal_form)

words_lemmatized
# print(words_lemmatized)

In [ ]:
from collections import Counter

Counter(words_lemmatized).most_common(10)
#print(Counter(words_lemmatized).most_common(10))

In [ ]:
import nltk

freq_bigramms = Counter(nltk.bigrams(words_lemmatized))
freq_bigramms.most_common(10)
#print(Counter(words_lemmatized).most_common(10))

# Лемматизатор MyStem

Сравнительная точность pymorphy и mystem для русского языка [здесь](http://web-corpora.net/wsgi/mystemplus.wsgi/mystemplus/compare_table/)


In [ ]:
!pip install pymystem3

In [2]:
from pymystem3 import Mystem
mystem = Mystem()

In [ ]:
mystem.lemmatize('mystem даже сам! умеет токенизировать текст')

In [ ]:
bananafish[:100]

In [ ]:
words_lemmatized_mystem = mystem.lemmatize(bananafish)
words_lemmatized_mystem
#print(words_lemmatized_mystem)

In [ ]:
from collections import Counter

Counter(words_lemmatized_mystem).most_common(10)
#print(Counter(words_lemmatized_mystem).most_common(10))

In [ ]:
# что-то пошло не так, как поправить?


In [ ]:
# найдите частотные слова


### морф. анализ в MyStem

In [ ]:
sometext = "Задача NLI важна для компьютерных лингвистов"
analyzed = mystem.analyze(sometext)
print(analyzed)

In [ ]:
# возвращает список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# wt - вес, увереность в правильности
# 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
print('Слово - ', analyzed[0]['text'])
print('Разбор слова - ', analyzed[0]['analysis'][0])
print('Лемма слова - ', analyzed[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', analyzed[0]['analysis'][0]['gr'])

In [ ]:
for word in mystem.analyze(sometext):
  if word.get('analysis'):
    print(word['analysis'][0]['lex'])

#[word['analysis'][0]['lex'] for word in mystem.analyze(sometext) if word.get('analysis')]

# Задание 1

Соберите 2 разные функции по лемматизации

В каждой пройдите весь путь от текста как строки до списка лемм

In [ ]:
def lemm_pymorphy(text):




In [ ]:
def lemm_mystem(text):




## Задание 2

Омонимия

In [ ]:
# омонимия
homonym1 = 'За время обучения я прослушал больше сорока курсов.'
homonym2 = 'Сорока своровала блестящее украшение со стола.'

# перед вами два предложения с омонимами: лемматизируйте их через pymorphy и mystem


In [ ]:
# pymorphy


In [ ]:
# mystem


In [ ]:
# Лемматизируйте через pymorphy и mystem
text = 'Гуляя в мужской парке по парку им.Карима Тинчурина в Казане, я увидел много белок. Я кармил их куриными яичками, но они не ели белок.'

In [ ]:
# pymorphy


In [ ]:
# mystem


In [ ]:
text = '''Варкалось. Хливкие шорьки
Пырялись по наве,
И хрюкотали зелюки,
Как мюмзики в мове.

О, бойся Бармаглота, сын!
Он так свирлеп и дик!
А в глу́ше ры́мит исполин —
Злопастный Брандашмыг!

Но взял он меч, и взял он щит,
Высоких полон дум.
В глущобу путь его лежит
Под дерево Тумтум.

Он стал под дерево и ждёт.
И вдруг граахнул гром —
Летит ужасный Бармаглот
И пылкает огнём!

Раз-два, раз-два! Горит трава,
Взы-взы — стрижает меч,
Ува! Ува! И голова
Барабардает с плеч!

О светозарный мальчик мой!
Ты победил в бою!
О храброславленный герой,
Хвалу тебе пою!

Варкалось. Хливкие шорьки
Пырялись по наве.
И хрюкотали зелюки,
Как мюмзики в мове.'''

In [ ]:
# pymorhy


In [ ]:
# mystem


Существуют и другие библиотеки для лемматизации

русский язык:

- [RNNmorph](https://github.com/IlyaGusev/rnnmorph)
- [deeppavlov](http://docs.deeppavlov.ai/en/master/)

другие языки:
- [UralicNLP](https://github.com/mikahama/uralicNLP)
- [hfst от Apertium](https://wiki.apertium.org/wiki/Hfst)
- [Stanza](https://stanfordnlp.github.io/stanza/)
- [SpaCy](https://spacy.io/usage/linguistic-features#morphology)
- [Trankit](https://trankit.readthedocs.io/en/latest/posdep.html)

### Маленькое доп. задание для тех, кто плохо разобрался

Предобработайте текст, который находится в переменной text, выполнив следующие этапы:

* Токенизация
* Приведение к нижнему регистру
* Удаление пунктуации
* Удаление стоп-слов
* Лемматизация

Многие действия вы умеете выполнять разными способами, выполните эти шаги так, как удобно вам

В конце у вас должен получиться список, содержащий слова из текста, в удобном для дальнейшей работы формате.



In [ ]:
text = """По результатам оценки критериев вузам присвоили один из рейтинговых статусов. Высшая школа экономики вошла в лидерскую группу А+, заняв первое место по таким критериям, как «Востребованность выпускников в найме», «Качество образовательной среды» и «Активность по развитию школьного образования».
Презентация рейтинга прошла в Москве на международной конференции AI Journey 2023. Один из основных критериев оценки вузов при расчете рейтинга — размер заработной платы молодых специалистов в течение года после завершения обучения (при трудоустройстве по специальности). В вузах-лидерах рейтинга средняя зарплата выпускников составила около 140 тыс. рублей.
В НИУ ВШЭ с 2021 года действует Центр искусственного интеллекта. Миссия Центра развитие и внедрение технологий искусственного интеллекта в разные сферы жизни человека и общества, отрасли науки и сектора экономики. НИУ ВШЭ стал одним из победителей конкурса на получение гранта от Правительства Российской Федерации для создания центра искусственного интеллекта.
Конкурс проводился в рамках федерального проекта «Искусственный интеллект», основная задача которого — стимулирование развития и внедрения технологий ИИ в России.
«По поручению главы государства Альянс в сфере ИИ совместно с Минобрнауки разработали рейтинг российских вузов по качеству подготовки специалистов по ИИ. Рейтинг является важнейшим индикатором качества образования в области ИИ и наглядно отражает мнение работодателей о том, насколько образовательные программы актуальны и отвечают запросу рынка. Среди 180 вузов, оказавшихся в рейтинге, 10 имеют оценки А+, А (хорошее качество) B+, B (приемлемое качество).
Абсолютные лидеры — это ВШЭ, МФТИ и ИТМО. Таким образом, ТОП-10 российских университетов уже могут конкурировать за звание лучших, а значит готовят высококвалифицированных специалистов и успешно развивают науку в области искусственного интеллекта», — отметил Дмитрий Чернышенко, заместитель председателя Правительства России, куратор нацпрограммы «Цифровая экономика».
Для объективной оценки качества специалистов по ИИ эксперты выделили 13 критериев, включая такие аспекты, как уровень заработных плат выпускников, их востребованность на рынке труда, наличие научных публикаций на конференциях и в журналах, количество призеров студенческих олимпиад, средний балл по ЕГЭ, и наличие программ, прошедших профессионально-общественную аккредитацию со стороны Альянса."""

In [ ]:
# Токенизация


In [ ]:
# Приведение к нижнему регистру


In [ ]:
# Удаление пунктуации


In [ ]:
# Удаление стоп-слов


In [ ]:
# Лемматизация


In [ ]:
# Можете собрать результат в функцию
